# 載入資料

In [1]:
import pickle
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
# 讀取檔案
with open('Apk.csv', 'rb') as file:
    data = pickle.load(file)

data

,Class,Permission,apk
0,Adware,"[android.permission.ACCESS_FINE_LOCATION, andr...",00325582d7caaf0f36ad333869df444a1aa39326e60745...
1,Adware,[com.android.launcher.permission.INSTALL_SHORT...,00567501fe3517b1bf6b363dd6c4d0972a77c591b0d7c2...
2,Adware,"[android.permission.READ_CONTACTS, android.per...",00621e015191863041e78726b863b7e1374b17fda69036...
3,Adware,[com.android.launcher.permission.INSTALL_SHORT...,0149b02468a8e582d76a7ad5094c15a4a07f4f08361ba5...
4,Adware,"[android.permission.ACCESS_FINE_LOCATION, andr...",018096f1e732c91fb315a84ab5d851430c5d2c124a7f8d...
...,...,...,...
2273,SMS,"[android.permission.RECEIVE_SMS, com.software....",ffa7d5600e82f89d4a93a068697336b824020fdc456f37...
2274,SMS,"[android.permission.RECEIVE_SMS, com.android.l...",ffa9c5c0b315995db25deea05ca1d9fcda925757c30f07...
2275,SMS,"[android.permission.RECEIVE_SMS, com.android.l...",ffd7935393ffb9c8a6fb1f3932a0fbcfd0295172bff3f4...
2276,SMS,"[android.permission.RECEIVE_SMS, com.software....",ffe183625bb11bdf21494d10cdaaaf1f4b5ea2e5748209...


In [2]:
import numpy as np
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer

labels = data['Permission']

# one-hot
mlb = MultiLabelBinarizer()
one_hot_labels = mlb.fit_transform(labels)


print("One-hot後的形狀:", one_hot_labels.shape)

One-hot後的形狀: (2278, 878)


# 做用來使用mlp random forest 的csv

In [3]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


labels = data['Permission']
classes = data['Class']


# 除了benign以外都設0，是benign則設1
binary_class = [1 if c == 'benign' else 0 for c in classes]

# 創 新 DataFrame
one_hot_df = pd.DataFrame(one_hot_labels, columns=mlb.classes_)
binary_class_df = pd.DataFrame({'BinaryClass': binary_class})

# 合併
merged_df = pd.concat([binary_class_df, one_hot_df], axis=1)

# 下載csv
merged_df.to_csv("f2.csv", index=False)


# 轉成圖片

In [5]:
import os

from tensorflow.keras.preprocessing.image import img_to_array
# 提取權限特徵
permissions = data['Permission']

# 步驟2: 進行多標籤的 One-Hot 編碼
mlb = MultiLabelBinarizer()
label_array = mlb.fit_transform(permissions)

# 步驟3: 轉換特徵值為圖像並保存到文件
image_size = (32, 64)  
images = []
output_dir = 'image_output'

# 創建存儲圖像的目錄
os.makedirs(output_dir, exist_ok=True)

for i in range(label_array.shape[0]):
    # 灰度
    image = Image.new('L', image_size)
    pixels = image.load()

    # 轉成圖像素
    for j in range(label_array.shape[1]):
        pixel_value = int(label_array[i][j] * 255)
        x = j % image_size[0]
        y = j // image_size[0]
        pixels[x, y] = pixel_value

    # 轉成numpy
    image_array = img_to_array(image)
    images.append(image_array)

    # 生成文件名
    filename = f"image_{i}.png"

    # 保存图像到文件
    image_path = os.path.join(output_dir, filename)
    image.save(image_path)


# CNN所需的csv

In [6]:
import os
import pandas as pd



image_output_folder = "image_output"
output_csv_file = "image2.csv"


result_df = pd.DataFrame(columns=['Class', 'Image'])


i=0
for apk_file in data['Class']:

    if(apk_file=='benign'):
        cla='1'
    else:
        cla='0'
    
    image_file = f"image_{i}.png" 

    result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
    i += 1

result_df.to_csv(output_csv_file, index=False)


C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed fro

C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed fro

C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed fro

C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed fro

C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed fro

C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed fro

C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'Class': cla, 'Image': image_file}, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_14124\862549304.py:23: FutureWarning: The frame.append method is deprecated and will be removed fro